In [2]:
import pandas as pd
import cobra.flux_analysis
import time
from functools import partial
from src.mp_functions import combinations_subset, parallelize_dataframe, knockout_FBA


"""A mess of a document with different code cells.
Good to to use for any testing that involves the Recon3D model as it takes some time to load in."""

start_time = time.time()
model_file_path = 'C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/Human-GEM.mat'
model = cobra.io.load_matlab_model(model_file_path)

end_time = time.time()
print('Model load time: %.6f seconds' % (end_time - start_time))

Academic license - for non-commercial use only - expires 2022-10-03
Using license file c:\gurobi\gurobi.lic
Model load time: 42.933011 seconds


In [3]:
SNP_results = pd.read_table('C:/Users/Sigve/Genome_Data/results/SNPs_non_synonymous.tsv', index_col=0)

# Prepare SNP combinations; lists genes to knock out by model id.
combinations = pd.read_table('C:/Users/Sigve/Genome_Data/results/SNP_combinations.tsv', index_col=0)
combinations['gene_ids'] = combinations['gene_ids'].apply(lambda x: x.split(';'))


In [4]:
# FBA code cell
start_time = time.time()


# Does not work if model imported as sbml for some reason
combinations = parallelize_dataframe(combinations, partial(combinations_subset, partial(knockout_FBA, model)), 8)

# Code for single thread FBA
#combinations['results'] = combinations['gene_ids'].apply(partial(knockout_FBA, model))


combinations.to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/test.tsv', sep='\t')

end_time = time.time()
print('FBA run time: %.6f seconds' % (end_time - start_time))

FBA run time: 46.810802 seconds


In [5]:
# Check for non nominal results
for i, result in combinations.iterrows():
    res = result['results'].objective_value
    if int(res) < 187:
        print(result)

SNP_ids     rs9827;rs4988323;rs547552367;rs182737113;rs190...
gene_ids    [ENSG00000078237, ENSG00000124789, ENSG0000011...
results                     <Solution 0.000 at 0x1ad370f3940>
Name: 1, dtype: object
SNP_ids     rs531900792;rs187915150;rs61752937;rs61753543;...
gene_ids    [ENSG00000133706, ENSG00000162434, ENSG0000012...
results                     <Solution 0.000 at 0x1ad3ad204f0>
Name: 4, dtype: object
SNP_ids     rs546148482;rs117145430;rs9997664;rs140078589;...
gene_ids    [ENSG00000110328, ENSG00000139370, ENSG0000012...
results                     <Solution 0.000 at 0x1ad3ad07e80>
Name: 7, dtype: object
SNP_ids     rs181476100;rs189865294;rs548917691;rs74583923...
gene_ids    [ENSG00000130816, ENSG00000031698, ENSG0000017...
results                     <Solution 0.000 at 0x1ad3ad09b20>
Name: 10, dtype: object
SNP_ids     rs138534343;rs149122082;rs140611183;rs56466485...
gene_ids    [ENSG00000010219, ENSG00000117543, ENSG0000015...
results                     <Solution 0

In [6]:
gene_lists = combinations['gene_ids'].to_list()
gene_lists = [ids for sublist in gene_lists for ids in sublist]

counts = pd.Series(gene_lists).value_counts()
counts

ENSG00000155657    18
ENSG00000159063    13
ENSG00000142875    13
ENSG00000185803    11
ENSG00000126883     9
                   ..
ENSG00000060762     1
ENSG00000198919     1
ENSG00000136872     1
ENSG00000075975     1
ENSG00000147548     1
Length: 537, dtype: int64

In [7]:
# Test stuff

#print(model.reactions.MAR04908.genes)
#print(model.reactions.MAR04908.gene_reaction_rule)
print(model.genes.get_by_id('ENSG00000103024'))

#model.genes.ENSG00000103024.knock_out()
gene = model.genes.get_by_id('ENSG00000103024')
gene


ENSG00000103024


Gene identifier,ENSG00000103024
Name,
Memory address,0x01ad3d74d7f0
Functional,False
In 26 reaction(s),"MAR07883, MAR04908, MAR04675, MAR04291, MAR04016, MAR04907, MAR04570, MAR04670, MAR07890, MAR04635, MAR04018, MAR04572, MAR07894, MAR06614, MAR07878, MAR04420, MAR04673, MAR06615, MAR04906,..."
